In [2]:
import weaviate
client = weaviate.Client(
    url="http://192.168.2.186:8080"
)

/usr/local/lib/python3.10/dist-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.7.1.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [36]:
import openai
import os
def get_embedding_openai(text, engine="text-embedding-3-large") : 
    Openai_API_KEY = ""
    os.environ["OPENAI_API_KEY"] =  Openai_API_KEY
    openai.api_key =os.getenv("OPENAI_API_KEY")

    # res = openai.Embedding.create(input=text,engine=engine)['data'][0]['embedding']
    from openai import OpenAI
    embedding_client = OpenAI()
    res= embedding_client.embeddings.create(input = text, model=engine).data[0].embedding
    return res

In [40]:
known_class=["b_with_title","law"]
def set_db_client():
    client = weaviate.Client(
        url="http://192.168.2.186:8080"
    )
    return client
def db_class_sync_check():
    global client
    global known_class
    print("--- Synchronizing vecotr DB ---")
    db_classes=list(map(lambda x : x["class"].lower(), client.schema.get()["classes"]))
    for dc in db_classes:
        if dc not in known_class:
            print(f"--- delte db class {dc} ---")
            client.schema.delete_class(dc)
            
def db_class_check():
    global client
    print("--- DB class check ---")
    db_classes=list(map(lambda x : x["class"].lower(), client.schema.get()["classes"]))
    return db_classes

def del_weaviate_class(class_name):
    global client
    print(f"--- delte db class {class_name} ---")
    client.schema.delete_class(class_name)
        
def create_weaviate(class_name):
    global client
    print("--- Create DB ---")
    class_obj = {
        "class": class_name,
        "vectorizer": "none",
    }
    client.schema.create_class(class_obj)

def save_weaviate(class_name,vectorizing_element,chunks):
    global client
    client.batch.configure(batch_size=100)
    print(f"--- Save DB (data size: {len(chunks)}) ---")
    with client.batch as batch:
        for i, chunk in enumerate(chunks):
            if i > 185000:
                if i%1000==0:print(f"{i}/{len(chunks)}")
                vector = get_embedding_openai(chunk[vectorizing_element])
                batch.add_data_object(data_object=chunk, class_name=class_name, vector=vector)
    print("--- Save DB DONE ---")

import json
def load_json(data_path):
    print("--- Load Data ---")
    with open(data_path) as f:
        data = json.load(f)
    return data

def db_processing(class_name,data_path,vectorizing_element):
    db_class_sync_check()
    db_class_list=db_class_check()
    if class_name in db_class_list: del_weaviate_class(class_name)
    create_weaviate(class_name)
    chunks=load_json(data_path)
    save_weaviate(class_name,vectorizing_element,chunks)

In [44]:
db_class_sync_check()
db_class_check()

--- Synchronizing vecotr DB ---
--- DB class check ---


['b_with_title']

In [41]:
db_processing("law","/workspace/.gen/Crawler/Law/law_final_data.json","law_content")

--- Synchronizing vecotr DB ---
--- DB class check ---
--- delte db class law ---
--- Create DB ---
--- Load Data ---


KeyboardInterrupt: 

In [43]:
del_weaviate_class("law")

--- delte db class law ---


In [10]:
property_list = list(map(lambda x: x["name"], client.schema.get("b_with_title")['properties']))
result = client.query.get("b_with_title", property_list).do()

In [11]:
result

{'data': {'Get': {'B_with_title': [{'source': 'https://finddme.github.io/natural%20language%20processing%20and%20linguistics/2021/01/16/TalLinzen2019/',
     'source_title': 'What Can Linguistics and Deep Learning Contribute to Each Other? (Linzen et al. (2019))\n      16 Jan 2021',
     'text': '예문 3번의 경우, that the parents love라는 관계절을 고려하여 주어와 동사의 수일치를 이룰 수 있는지를 볼 수 있다. 예문 4번의 경우는 anaphora의 distribution에 대해 알아보는 문장으로, that the architects like가 manager에 IP(Inflection Phrase), TP(tense phrase) 혹은 CP(Complementizer Phrase)로 귀속될 수 있기 때문에 architects는 뒤에 나오는 anaphora를 C-commend(성분통어)하지 못한다. 따라서 binding theory에 따라 architects와 수가 맞더라도 themselves가 올 경우 비문이 된다. 예문 5번은 NPI(부정극어)에 대한 distribution을 알아보는 문장이다. 이러한 문장은 모두 2009년에 사람을 대상으로 진행된 실험에서 가져온 문장으로, tagging이 전혀 되지 않은 raw 코퍼스에서는 찾아보기 힘들다. 이에 따라 tagging되지 않은 코퍼스로 훈련한 RNN모델에 위와 같은 문장으로 test를 돌려본 결과, 정확도가 극적으로 떨어지는 것을 확인할 수 있었다. 따라서 이러한 실험을 통해 모델의 취약점을 찾아 보완할 수 있다. 하지만 이와 같은 실험을 진행할 때 local heuristics를 방지해야 한다. 즉, 모델이 잔머리 굴리는 것을 방지해야 한다. (6) a. T